**This notebook is an exercise in the [Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/missing-values).**

---

Now it's your turn to test your new knowledge of **missing values** handling. You'll probably find it makes a big difference.

# Setup

The questions will give you feedback on your work. Run the following cell to set up the feedback system.

In this exercise, you will work with data from the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

Run the next code cell without changes to load the training and validation sets in `X_train`, `X_valid`, `y_train`, and `y_valid`.  The test set is loaded in `X_test`.

In [1]:
# Importing data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id')
X_test = pd.read_csv('../input/test.csv', index_col='Id')

**1.1 - Checking Data:**

In [2]:
# Shape train dataset:
print("Shape train dataset (com coluna target ):", X.shape)


# Shape test dataset:
print("Shape test dataset:", X_test.shape)


# Missing rows in target column train:
print("Missing rows in target column train:", (X.SalePrice.isnull().sum()))


# Numeric columns:
print("Numerical columns:", X.select_dtypes(include=np.number).shape[1])


# Missing rows in numeric columns:
#print("Numerical columns:", X[X.select_dtypes(include=np.number).isnull().sum()>0]

# Categorical columns:



# Missing rows in numeric columns:

Shape train dataset (com coluna target ): (1460, 80)
Shape test dataset: (1459, 79)
Missing rows in target column train: 0
Numerical columns: 37


**1.2 - Data split train/valid:**

In [3]:
# separate target from predictors
#X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)
print(X.shape)


# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=1)
print(X_train.shape)

(1460, 79)
(1168, 79)


**1.3 - Separação numerico e categorico:**

In [4]:
# Selecionando numericos
X_numerical_train = X_train.select_dtypes(exclude=['object'])
X_numerical_valid = X_valid.select_dtypes(exclude=['object'])
X_numerical_test = X_test.select_dtypes(exclude=['object'])

# Selecionando categoricos
X_categorical_train = X_train.select_dtypes(exclude=['int64', 'float64'])
X_categorical_valid = X_valid.select_dtypes(exclude=['int64', 'float64'])
X_categorical_test = X_test.select_dtypes(exclude=['int64', 'float64'])

print(X_numerical_train.shape)
print(X_numerical_valid.shape)
print(X_numerical_test.shape)
print(X_categorical_train.shape)
print(X_categorical_valid.shape)
print(X_categorical_test.shape) #Tudo certo, nenhum dado sumido.

(1168, 36)
(292, 36)
(1459, 36)
(1168, 43)
(292, 43)
(1459, 43)


**2.1 - Numericos: checar NAs**

In [5]:
# Quais colunas possuem NAs?
missing_val_count_by_column = (X_numerical_train.isnull().sum())
print("Colunas com NA tabela treino: \n", missing_val_count_by_column[missing_val_count_by_column > 0])

missing_val_count_by_column2 = (X_numerical_valid.isnull().sum())
print("Colunas com NA tabela validation: \n",missing_val_count_by_column2[missing_val_count_by_column2 > 0])


missing_val_count_by_column3 = (X_numerical_test.isnull().sum())
print("Colunas com NA tabela teste: \n",missing_val_count_by_column3[missing_val_count_by_column3 > 0])

Colunas com NA tabela treino: 
 LotFrontage    209
MasVnrArea       8
GarageYrBlt     61
dtype: int64
Colunas com NA tabela validation: 
 LotFrontage    50
GarageYrBlt    20
dtype: int64
Colunas com NA tabela teste: 
 LotFrontage     227
MasVnrArea       15
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt      78
GarageCars        1
GarageArea        1
dtype: int64


**2.2 - Inputation em colunas com NAs:**

In [6]:
# Approach será dividido em 2: 
    #Os Nas na coluna GarageYrBlt será preenchidos com os respectivos valores da coluna YearBuilt
    #Demais valores serão subsituidos pela média
    
X_numerical_train.GarageYrBlt = X_numerical_train.GarageYrBlt.fillna(X_numerical_train.YearBuilt)
X_numerical_valid.GarageYrBlt = X_numerical_valid.GarageYrBlt.fillna(X_numerical_valid.YearBuilt)
X_numerical_test.GarageYrBlt = X_numerical_test.GarageYrBlt.fillna(X_numerical_test.YearBuilt)

#Testando: 
    #Funciona - para retestar so colocar o codechunk acima aqui embaixo

# Inputando nas outras colunas:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
X_numerical_train_inp = pd.DataFrame(my_imputer.fit_transform(X_numerical_train))
X_numerical_valid_inp = pd.DataFrame(my_imputer.fit_transform(X_numerical_valid))
X_numerical_test_inp = pd.DataFrame(my_imputer.fit_transform(X_numerical_test))

    #recolocando nome colunas
X_numerical_train_inp.columns = X_numerical_train.columns
X_numerical_valid_inp.columns = X_numerical_valid.columns
X_numerical_test_inp.columns = X_numerical_test.columns

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


**2.2.1 - Checar:**

In [7]:
# TESTE PRA VER SE AINDA HÁ
missing_val_count_by_column = (X_numerical_train_inp.isnull().sum())
print("Colunas com NA tabela treino: \n", missing_val_count_by_column[missing_val_count_by_column > 0])

missing_val_count_by_column2 = (X_numerical_valid_inp.isnull().sum())
print("Colunas com NA tabela validation: \n",missing_val_count_by_column2[missing_val_count_by_column2 > 0])


missing_val_count_by_column3 = (X_numerical_test_inp.isnull().sum())
print("Colunas com NA tabela teste: \n",missing_val_count_by_column3[missing_val_count_by_column3 > 0])

print(X_numerical_train_inp.isnull().sum())

#Teste passado!!!

Colunas com NA tabela treino: 
 Series([], dtype: int64)
Colunas com NA tabela validation: 
 Series([], dtype: int64)
Colunas com NA tabela teste: 
 Series([], dtype: int64)
MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
dtype: int64


**3.1 - Categóricos: checar Nas:**

In [8]:
missing_val_count_by_column_cat1 = (X_categorical_train.isnull().sum())
print("Colunas com NA tabela treino: \n", missing_val_count_by_column_cat1[missing_val_count_by_column_cat1 > 0]) # 15 colunas

missing_val_count_by_column_cat2 = (X_categorical_valid.isnull().sum())
print("Colunas com NA tabela valid: \n", missing_val_count_by_column_cat2[missing_val_count_by_column_cat2 > 0]) # 13 colunas

missing_val_count_by_column_cat3 = (X_categorical_test.isnull().sum())
print("Colunas com NA tabela test: \n", missing_val_count_by_column_cat3[missing_val_count_by_column_cat3 > 0]) # 22 colunas

Colunas com NA tabela treino: 
 Alley           1097
MasVnrType         8
BsmtQual          30
BsmtCond          30
BsmtExposure      31
BsmtFinType1      30
BsmtFinType2      31
Electrical         1
FireplaceQu      548
GarageType        61
GarageFinish      61
GarageQual        61
GarageCond        61
PoolQC          1163
Fence            954
MiscFeature     1124
dtype: int64
Colunas com NA tabela valid: 
 Alley           272
BsmtQual          7
BsmtCond          7
BsmtExposure      7
BsmtFinType1      7
BsmtFinType2      7
FireplaceQu     142
GarageType       20
GarageFinish     20
GarageQual       20
GarageCond       20
PoolQC          290
Fence           225
MiscFeature     282
dtype: int64
Colunas com NA tabela test: 
 MSZoning           4
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinType2      42
KitchenQual        1
Functional   

In [9]:
# Teste para entender possiveis valores de cada uma das colunas

# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]


# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_categorical_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Alley', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('GarageFinish', 3),
 ('PavedDrive', 3),
 ('PoolQC', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('MasVnrType', 4),
 ('ExterQual', 4),
 ('BsmtQual', 4),
 ('BsmtCond', 4),
 ('BsmtExposure', 4),
 ('HeatingQC', 4),
 ('KitchenQual', 4),
 ('Fence', 4),
 ('MiscFeature', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('Electrical', 5),
 ('FireplaceQu', 5),
 ('GarageQual', 5),
 ('GarageCond', 5),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('BsmtFinType1', 6),
 ('BsmtFinType2', 6),
 ('Heating', 6),
 ('GarageType', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('Functional', 7),
 ('Condition2', 8),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 14),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

**3.2 - Inputation em colunas com NAs:**

 Para inputar as variaveis, vamos dividir em 2 approachs diferentes. 
*     Approach 1 - Há colunas que colocam NA quando a observação especifica tem valor 0 (EX: NA em alley é quando nao há alley naquela casa)
*     Approach 2 - Apos fazer isso, inputar lidando com dados nao disponiveis (bfill, ou most commom )

  Colunas categoricas que tem NA mas que nao é missing value:
*     1 -Alley - NA é No alley access
*     2 - BsmtQual - NA é no basement
*     3 - BsmtExposure - NA é No basement
*     4 - BsmtFinType1 - NA é No basement
*     5 - BsmtFinType2 - NA é No basement
*     6 - FireplaceQu - NA é No fireplace
*     7 - GarageType - NA é No Garage
*     8 - GarageFinish - NA é No Garage
*     9 - GarageQual - NA é No Garage
*     10 - GarageCond - NA é No Garage
*     11 - PoolQC - NA é No Pool
*     12 - Fence - NA é No Fence
*     13 - MiscFeature - NA é none
*     14 - BsmtCond - NA é No Basement

In [10]:
# columns where NaN values have meaning e.g. no pool etc.
cols_fillna = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu',
               'GarageQual','GarageCond','GarageFinish','MasVnrType','GarageType',
               'BsmtExposure','BsmtCond','BsmtQual','BsmtFinType1','BsmtFinType2']

# replace 'NaN' with 'None' in these columns

#Train
for col in cols_fillna:
    X_categorical_train[col].fillna('None',inplace=True)
    
#Valid
for col in cols_fillna:
    X_categorical_valid[col].fillna('None',inplace=True)

#Test
for col in cols_fillna:
    X_categorical_test[col].fillna('None',inplace=True)
    
    
    

# Checando
missing_val_count_by_column_cat1 = (X_categorical_train.isnull().sum())
print("Colunas com NA tabela treino: \n", missing_val_count_by_column_cat1[missing_val_count_by_column_cat1 > 0]) 

missing_val_count_by_column_cat2 = (X_categorical_valid.isnull().sum())
print("Colunas com NA tabela valid: \n", missing_val_count_by_column_cat2[missing_val_count_by_column_cat2 > 0])

missing_val_count_by_column_cat3 = (X_categorical_test.isnull().sum())
print("Colunas com NA tabela test: \n", missing_val_count_by_column_cat3[missing_val_count_by_column_cat3 > 0])

Colunas com NA tabela treino: 
 Electrical    1
dtype: int64
Colunas com NA tabela valid: 
 Series([], dtype: int64)
Colunas com NA tabela test: 
 MSZoning       4
Utilities      2
Exterior1st    1
Exterior2nd    1
KitchenQual    1
Functional     2
SaleType       1
dtype: int64


/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Fazendo assim, sobram apenas:
*  1 coluna (Eletrical) para estimar imputação em treino (1 valor)
*  0 coluna para estimar imputação em treino
*  7 colunas (MSZoning, Utilities, Exterior1st, Exterior2nd, KitchenQual, Functional, SaleType) para estimar imputação em treino (4,2,1,1,1,2,1 valores respectivamente)

In [11]:
# Inputando nas outras colunas:

my_imputer = SimpleImputer(strategy='most_frequent')
X_categorical_train_inp = pd.DataFrame(my_imputer.fit_transform(X_categorical_train))
X_categorical_valid_inp = pd.DataFrame(my_imputer.fit_transform(X_categorical_valid))
X_categorical_test_inp = pd.DataFrame(my_imputer.fit_transform(X_categorical_test))

    #recolocando nome colunas
X_categorical_train_inp.columns = X_categorical_train.columns
X_categorical_valid_inp.columns = X_categorical_valid.columns
X_categorical_test_inp.columns = X_categorical_test.columns

In [12]:
# Checando
missing_val_count_by_column_cat1 = (X_categorical_train_inp.isnull().sum())
print("Colunas com NA tabela treino: \n", missing_val_count_by_column_cat1[missing_val_count_by_column_cat1 > 0]) 

missing_val_count_by_column_cat2 = (X_categorical_valid_inp.isnull().sum())
print("Colunas com NA tabela valid: \n", missing_val_count_by_column_cat2[missing_val_count_by_column_cat2 > 0])

missing_val_count_by_column_cat3 = (X_categorical_test_inp.isnull().sum())
print("Colunas com NA tabela test: \n", missing_val_count_by_column_cat3[missing_val_count_by_column_cat3 > 0])

Colunas com NA tabela treino: 
 Series([], dtype: int64)
Colunas com NA tabela valid: 
 Series([], dtype: int64)
Colunas com NA tabela test: 
 Series([], dtype: int64)


**3.3 - Processamento One Hot Encoding:**

In [13]:
from sklearn.preprocessing import OneHotEncoder

# Use as many lines of code as you need!

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_categorical_train_inp))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_categorical_valid_inp))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_categorical_test_inp))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_categorical_train_inp.index
OH_cols_valid.index = X_categorical_valid_inp.index
OH_cols_test.index = X_categorical_test_inp.index

print(OH_cols_train.shape)
print(OH_cols_valid.shape)
print(OH_cols_test.shape) # todos tem mesma quantidade de linhas que a parte numerica

print(X_numerical_train_inp.shape)

(1168, 263)
(292, 263)
(1459, 263)
(1168, 36)


**4 - Juntar tabela categoricos com numericos:**

In [14]:
# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([X_numerical_train_inp, OH_cols_train], axis=1)
OH_X_valid = pd.concat([X_numerical_valid_inp, OH_cols_valid], axis=1)
OH_X_test = pd.concat([X_numerical_test_inp, OH_cols_test], axis=1)


print(OH_X_train.shape)
print(OH_X_valid.shape)
print(OH_X_test.shape)

(1168, 299)
(292, 299)
(1459, 299)


**5 - Modegalem:**

5.2 - Definição modelos:

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

#Definindo modelos:
model_1 = RandomForestRegressor(random_state=1)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='mae', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)
model_6 = XGBRegressor(random_state = 1)
model_7 = XGBRegressor(random_state = 123,n_jobs=2)

models = [model_1, model_2, model_3, model_4, model_5, model_6, model_7]

# Função que irá fazer fit, prever e retornar MAE
def score_model(model, X_t=OH_X_train, X_v=OH_X_valid, y_t=y_train, y_v=y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))

Model 1 MAE: 16594
Model 2 MAE: 16576
Model 3 MAE: 16640
Model 4 MAE: 17339
Model 5 MAE: 17584
Model 6 MAE: 16525
Model 7 MAE: 16525


In [16]:
# Fill in the line below: get test predictions
preds_test = model_6.predict(OH_X_test)

**6 - Submissão resultados:**

In [17]:

# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)